Test data
- attack dataframe 만들기

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [0]:
train = pd.read_pickle('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/test.pkl')

# Attack 추출
- session :0 -> attack 자원 주변 (일단 본진 멀티 나누지 말고, None으로 된 것만 처리)
- session :1 -> attack 일꾼
- session :2 -> attack 건물
- session :3 -> attack 본진
- session :4 -> 상대 진영 렐리 추출

In [0]:
train_camera_first=train.query('event==0')

In [4]:
# 상대 본진을 merge 시키기 위해 player 변경
train_camera_first.loc[train_camera_first['player']==0,
                       'player']=2
train_camera_first.loc[train_camera_first['player']==1,
                       'player']=3

train_camera_first.loc[train_camera_first['player']==2,
                       'player']=1
train_camera_first.loc[train_camera_first['player']==3,
                       'player']=0

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [0]:
train_camera_first=train_camera_first.groupby(['game_id','player']).first()

In [6]:
train=train.merge(train_camera_first['event_contents'],on=['game_id','player'])
train.head()

,game_id,time,player,species,event,event_contents_x,event_contents_y
0,38872.0,0,0.0,1.0,0.0,"at (22.25, 81.5078125)","at (120.25, 153.83984375)"
1,38872.0,0,0.0,1.0,4.0,NaN,"at (120.25, 153.83984375)"
2,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)"
3,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)"
4,38872.0,0,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)"


In [7]:
# 데이터 합칠 껍데기 생성
final_merge=pd.DataFrame(index=train.game_id.unique(),
                         data={'game_id':train.game_id.unique()})
final_merge.head()

,game_id
38872.0,38872.0
38873.0,38873.0
38874.0,38874.0
38875.0,38875.0
38876.0,38876.0


## Session :0
- Attack 자원 주변

In [8]:
attack_0 = train[train['event']==2]
attack_0.head()

,game_id,time,player,species,event,event_contents_x,event_contents_y
4,38872.0,0,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)"
25,38872.0,0,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)"
45,38872.0,1,0.0,1.0,2.0,(1542) - BuildAssimilator; Target: Destructibl...,"at (120.25, 153.83984375)"
49,38872.0,1,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)"
81,38872.0,2,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)"


In [9]:
attack_0=attack_0[attack_0['event_contents_x'].str.contains('Attack; Target: None')]
attack_0['event_contents_x']=0
attack_0.head()

,game_id,time,player,species,event,event_contents_x,event_contents_y
18635,38884.0,3,1.0,2.0,2.0,0,"at (22.75, 147.0078125)"
18772,38884.0,4,1.0,2.0,2.0,0,"at (22.75, 147.0078125)"
22577,38885.0,4,1.0,0.0,2.0,0,"at (163.25, 18.5078125)"
22596,38885.0,4,1.0,0.0,2.0,0,"at (163.25, 18.5078125)"
26507,38888.0,4,1.0,0.0,2.0,0,"at (24.75, 137.0078125)"


In [0]:
attack_0=attack_0.groupby(['game_id','player'])['event_contents_x'].count().reset_index()

player_0=attack_0[attack_0['player']==0]
player_1=attack_0[attack_0['player']==1]

In [11]:
#player_0
final_merge=final_merge.merge(player_0[['game_id','event_contents_x']],on='game_id',how='left').merge(player_1[['game_id','event_contents_x']],on='game_id',how='left')
final_merge.head()

,game_id,event_contents_x_x,event_contents_x_y
0,38872.0,NaN,NaN
1,38873.0,NaN,NaN
2,38874.0,NaN,NaN
3,38875.0,NaN,NaN
4,38876.0,NaN,NaN


In [12]:
final_merge.rename(columns={'event_contents_x_x':'0_자원공격',
                    'event_contents_x_y':'1_자원공격'},inplace=True)
final_merge.head()

,game_id,0_자원공격,1_자원공격
0,38872.0,NaN,NaN
1,38873.0,NaN,NaN
2,38874.0,NaN,NaN
3,38875.0,NaN,NaN
4,38876.0,NaN,NaN


## Session: 1
- attack 일꾼

In [13]:
attack_1 = train[(train['event']==2)|
                (train['event']==3)]
attack_1.head()

,game_id,time,player,species,event,event_contents_x,event_contents_y
2,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)"
3,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)"
4,38872.0,0,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)"
20,38872.0,0,0.0,1.0,3.0,"Location: (31.235595703125, 93.552734375, 49122)","at (120.25, 153.83984375)"
21,38872.0,0,0.0,1.0,3.0,"Location: (31.5546875, 93.2060546875, 49122)","at (120.25, 153.83984375)"


In [14]:
# 내 일꾼 파생변수 제작
attack_1.loc[attack_1['species'] ==0,'My_Unit'] = 'SCV'
attack_1.loc[attack_1['species'] ==1,'My_Unit'] = 'Probe'
attack_1.loc[attack_1['species'] ==2,'My_Unit'] = 'Drone'

attack_1.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,game_id,time,player,species,event,event_contents_x,event_contents_y,My_Unit
2,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)",Probe
3,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)",Probe
4,38872.0,0,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)",Probe
20,38872.0,0,0.0,1.0,3.0,"Location: (31.235595703125, 93.552734375, 49122)","at (120.25, 153.83984375)",Probe
21,38872.0,0,0.0,1.0,3.0,"Location: (31.5546875, 93.2060546875, 49122)","at (120.25, 153.83984375)",Probe


In [15]:
# event_2 만 출력
event_2 = attack_1[attack_1['event']==2]
event_2 = event_2[(event_2['event_contents_x'].str.contains('Target: SCV'))|
                  (event_2['event_contents_x'].str.contains('Target: Probe'))|
                  (event_2['event_contents_x'].str.contains('Target: Drone'))]
event_2.head()

,game_id,time,player,species,event,event_contents_x,event_contents_y,My_Unit
567,38872.0,7,0.0,1.0,2.0,(5A0) - Attack; Target: Probe [037C0001]; Loca...,"at (120.25, 153.83984375)",Probe
2338,38874.0,2,0.0,0.0,2.0,(5A0) - Attack; Target: SCV [03780001]; Locati...,"at (24.75, 128.0078125)",SCV
3021,38874.0,2,1.0,0.0,2.0,(5A0) - Attack; Target: SCV [037C0001]; Locati...,"at (151.25, 20.5078125)",SCV
3560,38875.0,1,1.0,1.0,2.0,(5A0) - Attack; Target: SCV [037C0001]; Locati...,"at (152.25, 142.0078125)",Probe
4549,38875.0,2,0.0,0.0,2.0,(5A0) - Attack; Target: Probe [035C0001]; Loca...,"at (31.75, 22.5078125)",SCV


In [0]:
player_0 = event_2[event_2['player']==0]
player_0=player_0.groupby('game_id')['My_Unit'].value_counts().reset_index(name='event2_0')

In [0]:
player_1 = event_2[event_2['player']==1]
player_1=player_1.groupby('game_id')['My_Unit'].value_counts().reset_index(name='event2_1')

In [18]:
final_merge=final_merge.merge(player_0[['game_id','event2_0']],on='game_id',how='left')\
.merge(player_1[['game_id','event2_1']],on='game_id',how='left')
final_merge[['event2_0','event2_1']]=final_merge[['event2_0','event2_1']].fillna(0)
final_merge.head()

,game_id,0_자원공격,1_자원공격,event2_0,event2_1
0,38872.0,NaN,NaN,1.0,0.0
1,38873.0,NaN,NaN,0.0,0.0
2,38874.0,NaN,NaN,1.0,1.0
3,38875.0,NaN,NaN,5.0,1.0
4,38876.0,NaN,NaN,0.0,0.0


In [19]:
#event_3
# event_3= attack_1[(attack_1['event']==3)&
#                   (attack_1['event_contents_x'].str.contains('Target: SCV'))|
#                   (attack_1['event_contents_x'].str.contains('Target: Probe'))|
#                   (attack_1['event_contents_x'].str.contains('Target: Drone'))]

event_3= attack_1[attack_1['event']==3]
event_3 = event_3[(event_3['event_contents_x'].str.contains('Target: SCV'))|
                  (event_3['event_contents_x'].str.contains('Target: Probe'))|
                  (event_3['event_contents_x'].str.contains('Target: Drone'))]



event_3['target_name']=event_3['event_contents_x'].apply(lambda x: x.split(';')[0])
event_3['target_location']=event_3['event_contents_x'].apply(lambda x: x.split(';')[1])
event_3.drop('event_contents_x',axis=1,inplace=True)

event_3['event_contents_y']=event_3['event_contents_y'].apply(lambda x: x[5:-1].split(','))
event_3.rename(columns={'event_contents_y':'enemy_camp'},inplace=True)
event_3['target_name']=event_3['target_name'].apply(lambda x: x.split('[')[0].split(':')[-1])
event_3['target_location']=event_3['target_location'].apply(lambda x: x.split('(')[-1].strip(')').split(','))

event_3['My_Unit'] = event_3['My_Unit'].apply(lambda x: x.strip())
event_3['target_name'] = event_3['target_name'].apply(lambda x: x.strip())

event_3.head()

# '''
# 서로 종족이 다를 때
# 어떻게 처리해줘야할지
# 고민점
# '''

,game_id,time,player,species,event,enemy_camp,My_Unit,target_name,target_location
563,38872.0,7,0.0,1.0,3.0,"[20.25, 153.83984375]",Probe,Probe,"[118.210693359375, 157.4345703125, 49120]"
3030,38874.0,2,1.0,0.0,3.0,"[51.25, 20.5078125]",SCV,SCV,"[20.463623046875, 129.3818359375, 49136]"
3652,38875.0,2,1.0,1.0,3.0,"[52.25, 142.0078125]",Probe,SCV,"[152.220458984375, 112.55029296875, 40912]"
4720,38875.0,2,0.0,0.0,3.0,"[1.75, 22.5078125]",SCV,Probe,"[155.3837890625, 114.79638671875, 40912]"
4807,38875.0,3,0.0,0.0,3.0,"[1.75, 22.5078125]",SCV,SCV,"[155.666748046875, 145.679443359375, 49120]"


In [20]:
player_0 = event_3[event_3['player']==0]
player_0.head()

,game_id,time,player,species,event,enemy_camp,My_Unit,target_name,target_location
563,38872.0,7,0.0,1.0,3.0,"[20.25, 153.83984375]",Probe,Probe,"[118.210693359375, 157.4345703125, 49120]"
4720,38875.0,2,0.0,0.0,3.0,"[1.75, 22.5078125]",SCV,Probe,"[155.3837890625, 114.79638671875, 40912]"
4807,38875.0,3,0.0,0.0,3.0,"[1.75, 22.5078125]",SCV,SCV,"[155.666748046875, 145.679443359375, 49120]"
4808,38875.0,3,0.0,0.0,3.0,"[1.75, 22.5078125]",SCV,SCV,"[155.76318359375, 145.73876953125, 49120]"
5020,38875.0,4,0.0,0.0,3.0,"[1.75, 22.5078125]",SCV,SCV,"[156.6298828125, 143.381591796875, 49120]"


In [0]:
# 내 종족과 상대 종족이 다른 경우
differ_unit= player_0[player_0['My_Unit'] != player_0['target_name']].groupby('game_id')['target_name'].value_counts().reset_index(name='count')
differ_unit
differ_unit = differ_unit.groupby('game_id')['count'].sum().reset_index()

In [22]:
# 내 종족과 상대 종족이 같은 경우
# 종족이 서로 같은 게임 인덱스만 출력
kk=train.groupby('game_id')['species'].nunique().reset_index()
same_index=list(kk[kk['species']==1]['game_id'].values)

# player 0만 출력
same_unit=player_0[player_0['game_id'].isin(same_index)]

same_unit['enemy_camp_x']= same_unit['enemy_camp'].apply(lambda x: float(x[0]))
same_unit['enemy_camp_y']= same_unit['enemy_camp'].apply(lambda x: float(x[1]))
same_unit['target_location_x']= same_unit['target_location'].apply(lambda x: float(x[0]))
same_unit['target_location_y']= same_unit['target_location'].apply(lambda x: float(x[1]))
same_unit.drop(['enemy_camp','target_location'],axis=1,inplace=True)

# 유클리디안 거리 30이하
same_unit['Euclid'] = np.sqrt((same_unit['enemy_camp_x']- same_unit['target_location_x'])**2+\
                              (same_unit['enemy_camp_y']- same_unit['target_location_y'])**2)
same_unit_30 = same_unit[same_unit['Euclid']<30]

same_unit_30=same_unit_30.groupby('game_id')['target_name'].value_counts().reset_index(name='count')
same_unit_30
same_unit_30 = same_unit_30.groupby('game_id')['count'].sum().reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas

In [0]:
final_merge=final_merge.merge(differ_unit[['game_id','count']],on='game_id',how='left')\
.merge(same_unit_30[['game_id','count']],on='game_id',how='left')

In [0]:
final_merge[['count_x','count_y']]=final_merge[['count_x','count_y']].fillna(0)
final_merge['0_일꾼공격']= final_merge['count_x']+ final_merge['count_y']

In [25]:
final_merge.drop(['count_x','count_y'],axis=1,inplace=True)
final_merge.head()

,game_id,0_자원공격,1_자원공격,event2_0,event2_1,0_일꾼공격
0,38872.0,NaN,NaN,1.0,0.0,0.0
1,38873.0,NaN,NaN,0.0,0.0,0.0
2,38874.0,NaN,NaN,1.0,1.0,0.0
3,38875.0,NaN,NaN,5.0,1.0,1.0
4,38876.0,NaN,NaN,0.0,0.0,0.0


In [26]:
player_1 = event_3[event_3['player']==1]
#player_1.head()

# 내 종족과 상대 종족이 다른 경우
differ_unit= player_1[player_1['My_Unit'] != player_1['target_name']].groupby('game_id')['target_name'].value_counts().reset_index(name='count')
differ_unit
differ_unit = differ_unit.groupby('game_id')['count'].sum().reset_index()

# 내 종족과 상대 종족이 같은 경우
# 종족이 서로 같은 게임 인덱스만 출력
kk=train.groupby('game_id')['species'].nunique().reset_index()
same_index=list(kk[kk['species']==1]['game_id'].values)

# player 1만 출력
same_unit= player_1[player_1['game_id'].isin(same_index)]

same_unit['enemy_camp_x']= same_unit['enemy_camp'].apply(lambda x: float(x[0]))
same_unit['enemy_camp_y']= same_unit['enemy_camp'].apply(lambda x: float(x[1]))
same_unit['target_location_x']= same_unit['target_location'].apply(lambda x: float(x[0]))
same_unit['target_location_y']= same_unit['target_location'].apply(lambda x: float(x[1]))
same_unit.drop(['enemy_camp','target_location'],axis=1,inplace=True)

# 유클리디안 거리 30이하
same_unit['Euclid'] = np.sqrt((same_unit['enemy_camp_x']- same_unit['target_location_x'])**2+\
                              (same_unit['enemy_camp_y']- same_unit['target_location_y'])**2)
same_unit_30 = same_unit[same_unit['Euclid']<30]

same_unit_30=same_unit_30.groupby('game_id')['target_name'].value_counts().reset_index(name='count')
same_unit_30 = same_unit_30.groupby('game_id')['count'].sum().reset_index()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/

In [27]:
final_merge=final_merge.merge(differ_unit[['game_id','count']],on='game_id',how='left')\
.merge(same_unit_30[['game_id','count']],on='game_id',how='left')

final_merge[['count_x','count_y']]=final_merge[['count_x','count_y']].fillna(0)
final_merge['1_일꾼공격']= final_merge['count_x']+ final_merge['count_y']

final_merge.drop(['count_x','count_y'],axis=1,inplace=True)
final_merge.head()

,game_id,0_자원공격,1_자원공격,event2_0,event2_1,0_일꾼공격,1_일꾼공격
0,38872.0,NaN,NaN,1.0,0.0,0.0,0.0
1,38873.0,NaN,NaN,0.0,0.0,0.0,0.0
2,38874.0,NaN,NaN,1.0,1.0,0.0,0.0
3,38875.0,NaN,NaN,5.0,1.0,1.0,1.0
4,38876.0,NaN,NaN,0.0,0.0,0.0,0.0


In [28]:
final_merge['Attack_worker_0'] = final_merge['event2_0'] + final_merge['0_일꾼공격']
final_merge['Attack_worker_1'] = final_merge['event2_1'] + final_merge['1_일꾼공격']
final_merge.drop(['event2_0','event2_1','0_일꾼공격','1_일꾼공격'],axis=1,inplace=True)
final_merge.head()

,game_id,0_자원공격,1_자원공격,Attack_worker_0,Attack_worker_1
0,38872.0,NaN,NaN,1.0,0.0
1,38873.0,NaN,NaN,0.0,0.0
2,38874.0,NaN,NaN,1.0,1.0
3,38875.0,NaN,NaN,6.0,2.0
4,38876.0,NaN,NaN,0.0,0.0


## Session: 3
- Attack 본진
- Target이 Location으로 잡혀있고 본진과 유클리드 30이내만

In [0]:
attack_3 = train[(train['event']==2)|
                (train['event']==3)]

In [30]:
attack_3.head()

,game_id,time,player,species,event,event_contents_x,event_contents_y
2,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)"
3,38872.0,0,0.0,1.0,3.0,"Target: None [01FC0001]; Location: (18.0, 87.5...","at (120.25, 153.83984375)"
4,38872.0,0,0.0,1.0,2.0,(15E0) - TrainProbe,"at (120.25, 153.83984375)"
20,38872.0,0,0.0,1.0,3.0,"Location: (31.235595703125, 93.552734375, 49122)","at (120.25, 153.83984375)"
21,38872.0,0,0.0,1.0,3.0,"Location: (31.5546875, 93.2060546875, 49122)","at (120.25, 153.83984375)"


In [0]:
event_2 = attack_3[attack_3['event'] ==2]

# SCV, Drone, Porbe를 뺀 모든 Attack 명령
event_2= event_2[(event_2['event_contents_x'].str.contains('Attack'))&
                 (event_2['event_contents_x'].str.contains('Location'))&
                 (event_2['event_contents_x'].str.contains('SCV')==False)&
                 (event_2['event_contents_x'].str.contains('Drone')==False)&
                 (event_2['event_contents_x'].str.contains('Probe')==False)]

In [32]:
event_2['target_location_x'] = event_2['event_contents_x'].apply(lambda x:float(x.split('Location:')[1][2:-8].split(',')[0]))
event_2['target_location_y'] = event_2['event_contents_x'].apply(lambda x:float(x.split('Location:')[1][2:-8].split(',')[1]))
event_2['enemy_camp_x']= event_2['event_contents_y'].apply(lambda x:float(x[4:-1].split(',')[0]))
event_2['enemy_camp_y']= event_2['event_contents_y'].apply(lambda x:float(x[4:-1].split(',')[1]))

event_2.drop(['event_contents_x','event_contents_y'],axis=1,inplace=True)

event_2.head()

,game_id,time,player,species,event,target_location_x,target_location_y,enemy_camp_x,enemy_camp_y
287,38872.0,4,0.0,1.0,2.0,17.560303,91.032715,120.25,153.839844
418,38872.0,5,0.0,1.0,2.0,42.000000,79.000000,120.25,153.839844
505,38872.0,6,0.0,1.0,2.0,146.497803,140.737793,120.25,153.839844
518,38872.0,6,0.0,1.0,2.0,143.500000,136.500000,120.25,153.839844
1212,38872.0,6,1.0,1.0,2.0,143.873779,131.881348,22.25,81.507812


In [33]:
event_2['player'].value_counts()

1.0    113688
0.0    112083
Name: player, dtype: int64

In [0]:
player_0 = event_2[event_2['player']==0]

In [35]:
# 유클리디안 거리 30이하
player_0['Euclid'] = np.sqrt((player_0['enemy_camp_x']- player_0['target_location_x'])**2+\
                             (player_0['enemy_camp_y']- player_0['target_location_y'])**2)

player_0_30 = player_0[player_0['Euclid']<30]
player_0_30
player_0_30=player_0_30.groupby('game_id')['player'].value_counts().reset_index(name='attack_enemy_camp_0')
player_0_30

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,game_id,player,attack_enemy_camp_0
0,38872.0,0.0,2
1,38875.0,0.0,4
2,38879.0,0.0,11
3,38880.0,0.0,4
4,38881.0,0.0,1
...,...,...,...
6289,55650.0,0.0,2
6290,55652.0,0.0,4
6291,55653.0,0.0,2
6292,55654.0,0.0,1


In [36]:
player_1 = event_2[event_2['player']==1]

# 유클리디안 거리 30이하
player_1['Euclid'] = np.sqrt((player_1['enemy_camp_x']- player_1['target_location_x'])**2+\
                             (player_1['enemy_camp_y']- player_1['target_location_y'])**2)

player_1_30 = player_1[player_1['Euclid']<30]
player_1_30
player_1_30=player_1_30.groupby('game_id')['player'].value_counts().reset_index(name='attack_enemy_camp_1')
player_1_30

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,game_id,player,attack_enemy_camp_1
0,38874.0,1.0,1
1,38875.0,1.0,2
2,38879.0,1.0,5
3,38880.0,1.0,1
4,38881.0,1.0,5
...,...,...,...
6387,55645.0,1.0,5
6388,55648.0,1.0,1
6389,55649.0,1.0,1
6390,55654.0,1.0,5


In [0]:
final_merge = final_merge.merge(player_0_30[['game_id','attack_enemy_camp_0']],on='game_id',how='left')\
.merge(player_1_30[['game_id','attack_enemy_camp_1']],on='game_id',how='left')

In [38]:
final_merge

,game_id,0_자원공격,1_자원공격,Attack_worker_0,Attack_worker_1,attack_enemy_camp_0,attack_enemy_camp_1
0,38872.0,NaN,NaN,1.0,0.0,2.0,NaN
1,38873.0,NaN,NaN,0.0,0.0,NaN,NaN
2,38874.0,NaN,NaN,1.0,1.0,NaN,1.0
3,38875.0,NaN,NaN,6.0,2.0,4.0,2.0
4,38876.0,NaN,NaN,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...
16782,55654.0,NaN,NaN,1.0,2.0,1.0,5.0
16783,55655.0,NaN,NaN,0.0,9.0,2.0,1.0
16784,55656.0,NaN,NaN,0.0,0.0,NaN,NaN
16785,55657.0,NaN,NaN,0.0,0.0,NaN,NaN


## Session:4
- 상대 진영으로 렐리 추출

In [0]:
attack_4 = train[(train['event']==2)|
                 (train['event']==3)]

In [40]:
event_2=attack_4[attack_4['event']==2]
event_2 = event_2[event_2['event_contents_x'].str.contains('SetRallyPoint')==True]

event_2['target_location_x'] = event_2['event_contents_x'].apply(lambda x:float(x.split('Location:')[1][2:-8].split(',')[0]))
event_2['target_location_y'] = event_2['event_contents_x'].apply(lambda x:float(x.split('Location:')[1][2:-8].split(',')[1]))
event_2['enemy_camp_x']= event_2['event_contents_y'].apply(lambda x:float(x[4:-1].split(',')[0]))
event_2['enemy_camp_y']= event_2['event_contents_y'].apply(lambda x:float(x[4:-1].split(',')[1]))

event_2.drop(['event_contents_x','event_contents_y'],axis=1,inplace=True)

event_2.head()

,game_id,time,player,species,event,target_location_x,target_location_y,enemy_camp_x,enemy_camp_y
7541,38878.0,2,0.0,1.0,2.0,159.215820,64.252686,163.25,18.507812
10293,38880.0,3,0.0,0.0,2.0,31.551025,121.509521,151.25,20.507812
10300,38880.0,3,0.0,0.0,2.0,30.479980,115.603027,151.25,20.507812
10458,38880.0,4,0.0,0.0,2.0,29.611572,115.341797,151.25,20.507812
10493,38880.0,4,0.0,0.0,2.0,30.957275,115.764648,151.25,20.507812


In [0]:
player_0 = event_2[event_2['player']==0]

In [42]:
# 유클리디안 거리 30이하
player_0['Euclid'] = np.sqrt((player_0['enemy_camp_x']- player_0['target_location_x'])**2+\
                             (player_0['enemy_camp_y']- player_0['target_location_y'])**2)

player_0_30 = player_0[player_0['Euclid']<30]
player_0_30
player_0_30=player_0_30.groupby('game_id')['player'].value_counts().reset_index(name='event2_Rally_enemy_camp_0')
player_0_30

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,game_id,player,event2_Rally_enemy_camp_0
0,39538.0,0.0,1
1,39634.0,0.0,1
2,40290.0,0.0,1
3,40496.0,0.0,1
4,41166.0,0.0,1
5,41607.0,0.0,1
6,42628.0,0.0,1
7,43473.0,0.0,1
8,45168.0,0.0,2
9,45578.0,0.0,2


In [0]:
player_1 = event_2[event_2['player']==1]

In [44]:
# 유클리디안 거리 30이하
player_1['Euclid'] = np.sqrt((player_1['enemy_camp_x']- player_1['target_location_x'])**2+\
                             (player_1['enemy_camp_y']- player_1['target_location_y'])**2)

player_1_30 = player_1[player_1['Euclid']<30]
player_1_30
player_1_30=player_1_30.groupby('game_id')['player'].value_counts().reset_index(name='event2_Rally_enemy_camp_1')
player_1_30

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,game_id,player,event2_Rally_enemy_camp_1
0,39019.0,1.0,1
1,39180.0,1.0,1
2,39263.0,1.0,2
3,40013.0,1.0,1
4,40578.0,1.0,1
5,40975.0,1.0,2
6,41873.0,1.0,1
7,41989.0,1.0,1
8,42137.0,1.0,1
9,42819.0,1.0,1


In [45]:
final_merge= final_merge.merge(player_0_30[['game_id','event2_Rally_enemy_camp_0']],on='game_id',how='left')\
.merge(player_1_30[['game_id','event2_Rally_enemy_camp_1']],on='game_id',how='left')

final_merge[['event2_Rally_enemy_camp_0','event2_Rally_enemy_camp_1']] = \
final_merge[['event2_Rally_enemy_camp_0','event2_Rally_enemy_camp_1']].fillna(0)

final_merge.head()

,game_id,0_자원공격,1_자원공격,Attack_worker_0,Attack_worker_1,attack_enemy_camp_0,attack_enemy_camp_1,event2_Rally_enemy_camp_0,event2_Rally_enemy_camp_1
0,38872.0,NaN,NaN,1.0,0.0,2.0,NaN,0.0,0.0
1,38873.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0
2,38874.0,NaN,NaN,1.0,1.0,NaN,1.0,0.0,0.0
3,38875.0,NaN,NaN,6.0,2.0,4.0,2.0,0.0,0.0
4,38876.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0


In [46]:
event_3=attack_4[attack_4['event']==3]
event_3= event_3[event_3['event_contents_x'].str.contains('Target:')==False]

event_3['target_location_x'] = event_3['event_contents_x'].apply(lambda x:float(x.split('Location:')[1][2:-8].split(',')[0]))
event_3['target_location_y'] = event_3['event_contents_x'].apply(lambda x:float(x.split('Location:')[1][2:-8].split(',')[1]))
event_3['enemy_camp_x']= event_3['event_contents_y'].apply(lambda x:float(x[4:-1].split(',')[0]))
event_3['enemy_camp_y']= event_3['event_contents_y'].apply(lambda x:float(x[4:-1].split(',')[1]))

event_3.drop(['event_contents_x','event_contents_y'],axis=1,inplace=True)

event_3.head()

,game_id,time,player,species,event,target_location_x,target_location_y,enemy_camp_x,enemy_camp_y
20,38872.0,0,0.0,1.0,3.0,31.235596,93.552734,120.25,153.839844
21,38872.0,0,0.0,1.0,3.0,31.554688,93.206055,120.25,153.839844
22,38872.0,0,0.0,1.0,3.0,114.534180,149.116211,120.25,153.839844
23,38872.0,0,0.0,1.0,3.0,130.183105,35.968994,120.25,153.839844
34,38872.0,0,0.0,1.0,3.0,28.919678,94.034180,120.25,153.839844


In [0]:
player_0 = event_3[event_3['player']==0]

In [48]:
# 유클리디안 거리 30이하
player_0['Euclid'] = np.sqrt((player_0['enemy_camp_x']- player_0['target_location_x'])**2+\
                             (player_0['enemy_camp_y']- player_0['target_location_y'])**2)

player_0_30 = player_0[player_0['Euclid']<30]
player_0_30
player_0_30=player_0_30.groupby('game_id')['player'].value_counts().reset_index(name='event3_Rally_enemy_camp_0')
player_0_30

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,game_id,player,event3_Rally_enemy_camp_0
0,38872.0,0.0,21
1,38873.0,0.0,3
2,38874.0,0.0,35
3,38875.0,0.0,68
4,38876.0,0.0,2
...,...,...,...
13919,55653.0,0.0,42
13920,55654.0,0.0,23
13921,55655.0,0.0,57
13922,55656.0,0.0,17


In [0]:
player_1 = event_3[event_3['player']==1]

In [50]:
# 유클리디안 거리 30이하
player_1['Euclid'] = np.sqrt((player_1['enemy_camp_x']- player_1['target_location_x'])**2+\
                             (player_1['enemy_camp_y']- player_1['target_location_y'])**2)

player_1_30 = player_1[player_1['Euclid']<30]
player_1_30
player_1_30=player_1_30.groupby('game_id')['player'].value_counts().reset_index(name='event3_Rally_enemy_camp_1')
player_1_30

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,game_id,player,event3_Rally_enemy_camp_1
0,38872.0,1.0,67
1,38873.0,1.0,6
2,38875.0,1.0,65
3,38878.0,1.0,7
4,38879.0,1.0,37
...,...,...,...
13996,55653.0,1.0,51
13997,55654.0,1.0,49
13998,55655.0,1.0,21
13999,55656.0,1.0,104


In [51]:
final_merge= final_merge.merge(player_0_30[['game_id','event3_Rally_enemy_camp_0']],on='game_id',how='left')\
.merge(player_1_30[['game_id','event3_Rally_enemy_camp_1']],on='game_id',how='left')

final_merge[['event3_Rally_enemy_camp_0','event3_Rally_enemy_camp_1']] = \
final_merge[['event3_Rally_enemy_camp_0','event3_Rally_enemy_camp_1']].fillna(0)

final_merge.head()

,game_id,0_자원공격,1_자원공격,Attack_worker_0,Attack_worker_1,attack_enemy_camp_0,attack_enemy_camp_1,event2_Rally_enemy_camp_0,event2_Rally_enemy_camp_1,event3_Rally_enemy_camp_0,event3_Rally_enemy_camp_1
0,38872.0,NaN,NaN,1.0,0.0,2.0,NaN,0.0,0.0,21.0,67.0
1,38873.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,3.0,6.0
2,38874.0,NaN,NaN,1.0,1.0,NaN,1.0,0.0,0.0,35.0,0.0
3,38875.0,NaN,NaN,6.0,2.0,4.0,2.0,0.0,0.0,68.0,65.0
4,38876.0,NaN,NaN,0.0,0.0,NaN,NaN,0.0,0.0,2.0,0.0


In [0]:
final_merge = final_merge.fillna(0)

In [0]:
final_merge.to_csv('/content/drive/My Drive/Colab Notebooks/data/데이콘/스타2/attack_test.csv')

In [55]:
final_merge['game_id'].value_counts()

52743.0    1
44151.0    1
46085.0    1
42508.0    1
53267.0    1
          ..
51826.0    1
48377.0    1
54912.0    1
40272.0    1
38912.0    1
Name: game_id, Length: 16787, dtype: int64